In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [37]:
train_df = pd.read_csv('../data/cleaned_train.csv')

# Extract the columns you want to use as input features
columns = ['BERT_sentiment_score', 'normalised_word_count', 'pub_day', 'pub_hour']
# Extract input features from the dataframe
x_train = train_df[columns].dropna()

y_train = np.array(train_df['n_comments'])

In [38]:
test_df = pd.read_csv('../data/cleaned_test.csv')

x_test = test_df[columns].dropna()

In [39]:
def label_encode(raw_data):
    # Concatenate train and test data vertically to ensure consistent label encoding
    combined_df = pd.concat([train_df['topic'], test_df['topic']], axis=0)

    # Create an instance of LabelEncoder
    le = LabelEncoder()

    # Fit and transform the combined data using LabelEncoder
    combined_encoded = le.fit_transform(combined_df)
    
    return np.array(le.transform(raw_data.dropna()))

In [40]:
# Apply the LabelEncoder transformation on train data
train_topic_encoded = pd.DataFrame(label_encode(train_df['topic']), columns=['topic encoded'])
# Concatenate the 'topic_encoded' tensor with the other input features
x_train = pd.concat([x_train, train_topic_encoded], axis=1)

In [41]:
# Apply the LabelEncoder transformation on train data
test_topic_encoded = pd.DataFrame(label_encode(test_df['topic']), columns=['topic encoded'])
# Concatenate the 'topic_encoded' tensor with the other input features
x_test = pd.concat([x_test, test_topic_encoded], axis=1)

In [42]:
x_train

,BERT_sentiment_score,normalised_word_count,pub_day,pub_hour,topic encoded
0,0.0,0.043292,2.0,0.0,48
1,4.0,0.059366,2.0,3.0,18
2,4.0,0.067435,2.0,5.0,56
3,4.0,0.073775,2.0,5.0,56
4,0.0,0.071470,2.0,9.0,40
...,...,...,...,...,...
12456,0.0,0.082805,2.0,18.0,26
12457,3.0,0.005379,2.0,20.0,48
12458,0.0,0.063208,2.0,21.0,48
12459,0.0,0.109190,2.0,23.0,76


In [43]:
x_test

,BERT_sentiment_score,normalised_word_count,pub_day,pub_hour,topic encoded
0,0,0.058982,3,0,48
1,0,0.057509,3,0,48
2,2,0.090170,3,0,48
3,2,0.054115,3,2,18
4,1,0.172014,3,4,53
...,...,...,...,...,...
3868,2,0.063913,3,15,48
3869,4,0.070189,3,15,9
3870,0,0.091258,3,16,13
3871,2,0.083445,3,17,81


In [44]:
# Create MLP model
model = Sequential()
num_features = 5
model.add(Dense(64, activation='relu', input_dim=num_features))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [45]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [46]:
# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
312/312 [==============================] - 1s 2ms/step - loss: 301130.9062 - val_loss: 241812.5312
Epoch 2/100
312/312 [==============================] - 1s 2ms/step - loss: 276505.7812 - val_loss: 238440.5156
Epoch 3/100
312/312 [==============================] - 1s 2ms/step - loss: 271604.8438 - val_loss: 236466.2969
Epoch 4/100
312/312 [==============================] - 1s 2ms/step - loss: 266051.9375 - val_loss: 234099.0312
Epoch 5/100
312/312 [==============================] - 1s 2ms/step - loss: 261721.8594 - val_loss: 231310.7812
Epoch 6/100
120/312 [==========>...................] - ETA: 0s - loss: 253104.4375

KeyboardInterrupt: 

In [72]:
# save the trained model
model.save('MLP_prediction_model.h5')

In [76]:
y_pred = model.predict(x_test)

122/122 [==============================] - 0s 1ms/step


In [77]:
y_pred[0]

array([934.73047], dtype=float32)